# 載入套件

In [1]:
#!pip install tensorflow
#!pip install xgboost

In [2]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import tensorflow
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score,confusion_matrix

In [7]:
pd.set_option('display.max_columns', None)
df=pd.read_csv('0416_機器學習.csv')
from sklearn.utils import shuffle
df=shuffle(df)


In [8]:
df=df.drop('department_name',axis=1) #沒刪乾淨 之後確定
df=df.drop('fashion_news_frequency',axis=1)
df=df.drop('FN',axis=1)
df=df.drop('Active',axis=1)
df=df.drop('club_member_status',axis=1)

# 開始訓練

In [9]:
from sklearn.preprocessing import LabelEncoder #需要特徵都轉數值
labelencoder = LabelEncoder()

df['product_group_name']= labelencoder.fit_transform(df['product_group_name'])
df['graphical_appearance_name']= labelencoder.fit_transform(df['graphical_appearance_name'])
df['colour_group_name']= labelencoder.fit_transform(df['colour_group_name'])
df['section_name']= labelencoder.fit_transform(df['section_name'])
df['garment_group_name']= labelencoder.fit_transform(df['garment_group_name'])
#以下實驗
df['Year']= labelencoder.fit_transform(df['Year'])
df['pricelabel']= labelencoder.fit_transform(df['pricelabel'])
df['season']= labelencoder.fit_transform(df['season'])
df['salelabel']= labelencoder.fit_transform(df['salelabel'])

df.head()

,article_id,sales_channel_id,product_group_name,graphical_appearance_name,colour_group_name,section_name,garment_group_name,Year,season,pricelabel,salelabel
61584,739590032,2,7,16,20,26,14,2,1,1,0
35968,792817001,2,8,19,1,22,17,1,0,1,0
23537,811925002,2,7,0,1,26,14,2,2,3,0
87156,717664004,1,3,19,40,27,6,0,3,2,1
92125,766346004,2,2,19,24,20,15,2,1,2,1


In [10]:
sdf=df

In [11]:
#類型轉換瘦身 使機器學習運算更快
sdf['garment_group_name']=sdf['garment_group_name'].astype('int32')
sdf['section_name']=sdf['section_name'].astype('int32')
sdf['colour_group_name']=sdf['colour_group_name'].astype('int32')
sdf['graphical_appearance_name']=sdf['graphical_appearance_name'].astype('int32')
sdf['product_group_name']=sdf['product_group_name'].astype('int32')
sdf['sales_channel_id']=sdf['sales_channel_id'].astype('int32')
sdf['article_id']=sdf['article_id'].astype('int32')

sdf['Year']=sdf['Year'].astype('int32')
sdf['season']=sdf['season'].astype('int32')
sdf['pricelabel']=sdf['pricelabel'].astype('int32')
sdf['salelabel']=sdf['salelabel'].astype('int32')

sdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95084 entries, 61584 to 75721
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   article_id                 95084 non-null  int32
 1   sales_channel_id           95084 non-null  int32
 2   product_group_name         95084 non-null  int32
 3   graphical_appearance_name  95084 non-null  int32
 4   colour_group_name          95084 non-null  int32
 5   section_name               95084 non-null  int32
 6   garment_group_name         95084 non-null  int32
 7   Year                       95084 non-null  int32
 8   season                     95084 non-null  int32
 9   pricelabel                 95084 non-null  int32
 10  salelabel                  95084 non-null  int32
dtypes: int32(11)
memory usage: 4.7 MB


In [12]:
#設定要預測的Y
labels=sdf['article_id'] 

In [13]:
#移除要預測的Y 表就只剩下X
sdf=sdf.drop('article_id',axis=1) 

In [14]:
#訓練集、測試集分割
import tensorflow as tf 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sdf, labels,test_size=0.2, random_state=80) 

# xgboost

In [3]:
from xgboost.sklearn import XGBClassifier
xgbc = XGBClassifier(        
        n_estimators=8,     #樹的個數
        learning_rate= 0.1,    # 如同學習率 
        max_depth=15,       # 構建樹的深度，越大越容易過擬合    
        subsample=1,         # 隨機取樣訓練樣本 訓練例項的子取樣比
        gamma=0,         # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
        reg_lambda=1,          # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
        max_delta_step=0,        #最大增量步長，我們允許每個樹的權重估計。
        colsample_bytree=1,         # 生成樹時進行的列取樣 
        min_child_weight=1, 
                     # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
                     # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
                     #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
        seed=1000         #隨機種子
#        reg_alpha=0,        # L1 正則項引數
        #scale_pos_weight=1,        #如果取值大於0的話，在類別樣本不平衡的情況下有助於快速收斂。平衡正負權重
        #objective= 'multi:softmax',         #多分類的問題 指定學習任務和相應的學習目標
        #num_class=10,        # 類別數，多分類與 multisoftmax 並用
#        silent=0 ,        # 設定成1則沒有執行資訊輸出，最好是設定為0.是否在執行升級時列印訊息。
#        nthread=4,        # cpu 執行緒數 預設最大
        #eval_metric= 'auc'
)

xgbc.fit(X_train, y_train) 



NameError: name 'X_train' is not defined

In [5]:

import pickle
# Fit the model on training set
# save the model to disk

# 將訓練模型轉pickle 
from xgboost.sklearn import XGBClassifier
xgbc = XGBClassifier(        
        n_estimators=8,     #樹的個數
        learning_rate= 0.1,    # 如同學習率 
        max_depth=15,       # 構建樹的深度，越大越容易過擬合    
        subsample=1,         # 隨機取樣訓練樣本 訓練例項的子取樣比
        gamma=0,         # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
        reg_lambda=1,          # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
        max_delta_step=0,        #最大增量步長，我們允許每個樹的權重估計。
        colsample_bytree=1,         # 生成樹時進行的列取樣 
        min_child_weight=1, 
        seed=1000         #
)
MODEL_x2article = 'MODEL_x2article.sav'
pickle.dump(xgbc, open(MODEL_x2article, 'wb'))



# 從這開始 由視覺化網站的python 檔執行


In [20]:
# 載入機器學習已經訓練好的模型
with open('MODEL_x2article.sav', 'rb') as f:
    loaded_model = pickle.load(f)
    XGB_pred=loaded_model.predict(X_test) #模型去預測X_test
    print( "網頁需要輸入的X:",X_test)

    print("網頁預測結果:",XGB_pred) #此為根據X_test預測出的Y

NotFittedError: need to call fit or load_model beforehand

In [ ]:
XGBtrain_score=xgbc.score(X_train, y_train)
print(XGBtrain_score) #X與Y訓練集的準確率




In [ ]:
pickle.dump(xgbc, open('MODEL_x2article.pkl','wb'))


In [ ]:
# XGBtest_score=xgbc.score(X_test,y_test)
XGBtest_score = loaded_model.score(X_test, y_test)

print('The Accuracy of XGBOOST Classifier on testing set:', XGBtest_score) #X與Y測試集的準確率

In [ ]:
# 達到一定標準證明模型準確可以使用
# 只要再製造X_test 就可以print(XGB_pred)得到要的Y

In [ ]:
#建立X_test架構====想法
#填入網頁輸入內容連接的函式?

請輸入購買管道=pd.Series('線上') #此為透過網頁輸入的資料
請輸入想要類別=pd.Series('上衣') #此為透過網頁輸入的資料
請輸入想要樣式=pd.Series('圓形') #此為透過網頁輸入的資料

TestX=pd.DataFrame({'sales_channel_id':請輸入購買管道,
                   'product_group_name':請輸入想要類別,
                   'graphical_appearance_name':請輸入想要樣式})
TestX

In [ ]:
#以上方想法請參考一條龍的檔案